##TODO: Introdução

In [ ]:
## Install Required Packs
# !pip install pandas
# !pip install numpy
# !pip install statsmodels
# !pip install matplotlib
# !pip install -U seaborn
# !pip install pingouin
# !pip install statstests
# !pip install scipy
# !pip install openpyxl

In [ ]:
## Import Required Packs
import pandas as pd
import statsmodels.api as sm
from Utils.VariablesExtensions import VariablesExtensions
from Utils.GraphTools import GraphTools

In [ ]:
## Load dataset (Source: https://www.kaggle.com/datasets/elakiricoder/jiffs-house-price-prediction-dataset)
house_dataframe = pd.read_excel("../Datasets/house_price.xlsx")

# Get Metric Variables
metric_indexes = [
  'land_size_sqm',
  'house_size_sqm',
  'no_of_rooms',
  'no_of_bathrooms',
  'distance_to_school',
  'house_age',
  'distance_to_supermarket_km',
  'crime_rate_index',
  'property_value']

metric_variables = house_dataframe[metric_indexes].describe()

print(metric_variables)

In [ ]:
## Get Nominal Variables 
nominal_indexes = [
  'large_living_room',
  'parking_space',
  'front_garden',
  'swimming_pool',
  'wall_fence',
  'water_front',
  'room_size_class']

VariablesExtensions.print_nominal_counts(
  dataFrame=house_dataframe,
  indexes=nominal_indexes)

## Binarization of nominal variables
dummified_dataframe = pd.get_dummies(
  data=house_dataframe, 
  columns=nominal_indexes, 
  drop_first=True,
  dtype='float')

In [ ]:
target_index = 'property_value'
expresion = VariablesExtensions.get_regression_command_string(
  dataFrame=dummified_dataframe,
  target_index=target_index)
print(expresion)

In [ ]:
## Calculate model using multiple linear regression
model = sm.OLS.from_formula(
  formula = expresion,
  data=dummified_dataframe).fit()
model.summary()

In [ ]:
## Analyzing the model using the sample observations
model_analysis = pd.DataFrame()
model_analysis['sample_values'] = house_dataframe[target_index]
model_analysis['fitted_values'] = model.fittedvalues
model_analysis['residual_values'] = model.resid
model_analysis['residual_percent'] = (model.resid/house_dataframe[target_index]) * 100

#print(model_analysis)

GraphTools.generate_sample_fitted_analysis_graph(
  dataFrame=model_analysis,
  target_index_name='sample_values',
  fitted_index_name='fitted_values',
  title='Graphical Analysis - Fitted vs. Sample values'
)

#%% Coeficientes

# Note que o 'house_size_sqm' e 'no_of_rooms' agora têm impactos positivos

#%% Realizando predições para outras observações (modelo original)

# Qual é o preço médio estimado para uma casa com:
    # tamanho do terreno (m²) = 350
    # tamanho da casa (m²) = 200
    # quantidade de quartos = 3
    # quantidade de banheiros = 3
    # distância até a escola (km) = 4.5
    # idade da casa (anos) = 5
    # distância até o mercado (km) = 1.0
    # indicador de criminalidade da região = 1.20
    # sala ampla: sim
    # garagem: sim
    # jardim frontal: sim
    # piscina: sim
    # cercado: não
    # vista para lagos/rios: não
    # classificação dos quartos: classe 2

In [ ]:
# Utilizando a função "predict"
values_to_predit = pd.DataFrame({ 'land_size_sqm': [350],
                                  'house_size_sqm': [200], 
                                  'no_of_rooms': [3], 
                                  'no_of_bathrooms': [3],
                                  'distance_to_school': [4.5], 
                                  'house_age': [5], 
                                  'distance_to_supermarket_km': [1.0],
                                  'crime_rate_index': [1.2], 
                                  'large_living_room_Yes': [1],
                                  'parking_space_Yes': [1], 
                                  'front_garden_Yes': [1], 
                                  'swimming_pool_Yes': [1],
                                  'wall_fence_Yes': [0],
                                  'water_front_Yes': [0], 
                                  'room_size_class_Three': [0],
                                  'room_size_class_Two': [1], 
                                  'room_size_class_Zero': [0]})

predict_property = model.predict(values_to_predit) 
#print(predict_property)

currency_convention = round((predict_property[0]),2)
print(f'Value predicted by the model: ${currency_convention}')

#%% Alguns coeficientes podem apresentar uma interpretação "inesperada"

# Exemplos:
    ## quanto maior o tamanho da casa, menor o preço (ceteris paribus)
    ## quanto mais quartos, menor o preço (ceteris paribus)
    
# É importante notar que os resultados são interpretados de forma multivariada
    ## o resultado ocorre na presença das demais variáveis
    
#%% Matriz de correlações de Pearson entre variáveis métricas

In [ ]:
corretation_matrix = house_dataframe[metric_indexes].corr()
##print(metric_indexes_correlate)

GraphTools.generate_pearson_correlation_graph(
  dataFrame=corretation_matrix,
  title='Pearson correlations')

#%% Criando nova da fórmula

# 'land_size_sqm' tem correlação elevada com 'house_size_sqm' e 'no_of_rooms'
# Vamos remover o tamanho do terreno do modelo

In [ ]:
index_to_remove = 'land_size_sqm'

new_expresion = VariablesExtensions.get_regression_command_string(
  dataFrame=dummified_dataframe,
  target_index=target_index,
  columns_to_remove=[index_to_remove])

new_model = sm.OLS.from_formula(formula = new_expresion,
                                data=dummified_dataframe).fit()
new_model.summary()